# MAS - PROJECT

# Brench and bound DSF

Lib

In [71]:
import random
import numpy as np
import itertools
import time

list and tuples

In [72]:
# Function to perform the union operation
def union_listOflists_list(X, o_d):
    # Convert lists to tuples
    X_tuples = set(map(tuple, X))
    o_d_tuple = tuple(o_d)
    
    # Perform the union operation
    result_set = X_tuples | {o_d_tuple}
    
    # Convert the result back to a list of lists
    result = [list(item) for item in result_set]
    
    return result

# Example lists
l1 = ['a', 'b']
l2 = ['c', 'd']
l3 = ['e', 'f']
l4 = ['g', 'h']

# Example X and O
X = [l1, l2, l3]
O = [l1, l4]  # Suppose O[0] = l1, O[1] = l4

# Example usage
d = 0  # Suppose we want to use the first element of O
result = union_listOflists_list(X, O[d])
print(result)  # Output: [['a', 'b'], ['c', 'd'], ['e', 'f']] (since O[0] = l1 is already in X)

d = 1  # Suppose we want to use the second element of O
result = union_listOflists_list(X, O[d])
print(result)  # Output: [['a', 'b'], ['c', 'd'], ['e', 'f'], ['g', 'h']] (since O[1] = l4 is not in X)


[['e', 'f'], ['c', 'd'], ['a', 'b']]
[['e', 'f'], ['c', 'd'], ['a', 'b'], ['g', 'h']]


# o_X ecc... 

$
o_X \longrightarrow o_X (\theta)
$

In [73]:
def o_X(theta, X, g, u):
    """
    This function returns the element o in X that maximizes g(theta, o) 
    subject to the condition that u(theta, o) >= u(theta, o') for all o' in X.
    
    :param g: Function g(θ, o)
    :param X: List of elements
    :param u: Function u(θ, o)
    :param theta: Parameter θ
    
    :return: The element in X that maximizes g(θ, o) under the given condition
    """
    # Finding the element that satisfies the condition on u and maximizes g
    max_element = None
    max_value = float('-inf')
    
    for o in X:
        if all(u(theta, o) >= u(theta, o_prime) for o_prime in X):
            current_value = g(theta, o)
            if current_value > max_value:
                max_value = current_value
                max_element = o
    
    return max_element


$
v \longrightarrow v(X)
$ 

In [74]:
def v_X(X, Theta, p, g, u):
    """
    This function computes the value of v(X) given by:
    sum_{theta in Theta} p(theta) * g(theta, o_X(theta))
    
    :param X: List of elements
    :param Theta: List of possible values for theta
    :param p: Probability distribution function over Theta
    :param g: Function g(theta, o)
    :param u: Function u(theta, o)
    
    :return: The computed value of v(X)
    """
    # Compute the sum
    v = sum(p(theta) * g(theta, o_X(theta, X, g, u)) for theta in Theta)
    
    return v

$
o_{X,Y} \longrightarrow o_{X,Y} (\theta)
$

In [75]:
def o_XY(theta, X, Y, O, g, u):
    """
    This function returns the element o in X that maximizes g(theta, o) 
    subject to the condition that u(theta, o) >= u(theta, o') for all o' in X.
    
    :param g: Function g(θ, o)
    :param X: List of elements
    :param u: Function u(θ, o)
    :param theta: Parameter θ
    
    :return: The element in X that maximizes g(θ, o) under the given condition
    """
    # Finding the element that satisfies the condition on u and maximizes g
    max_element = None
    max_value = float('-inf')

    # Convert elements of O and Y to tuples for comparison
    O_tuples = [tuple(o) for o in O]
    Y_tuples = [tuple(y) for y in Y]
    # Compute the difference using list comprehension
    O_minus_Y = [list(o) for o in O_tuples if o not in Y_tuples]
    
    for o in O_minus_Y: 
        if all(u(theta, o) >= u(theta, o_prime) for o_prime in X):
            current_value = g(theta, o)
            if current_value > max_value:
                max_value = current_value
                max_element = o
    
    return max_element


$
v_{XY} 
$

In [76]:
def v_XY(X, Y, O, Theta, p, g, u):
    """
    This function computes the value of v(X) given by:
    sum_{theta in Theta} p(theta) * g(theta, o_X(theta))
    
    :param X: List of elements
    :param Theta: List of possible values for theta
    :param p: Probability distribution function over Theta
    :param g: Function g(theta, o)
    :param u: Function u(theta, o)
    
    :return: The computed value of v(X)
    """
    # Compute the sum
    v = sum(p(theta) * g(theta, o_XY(theta, X, Y, O,  g, u)) for theta in Theta)
    
    return v

# Brench and bound deep search

In [77]:
class SearchAlgorithm:
    def __init__(self, O, Theta, p, o_XY, v_XY, g, u, L = float('-inf'), CB = set()):
        """
        Initialize the SearchAlgorithm object.
        """
        self.O = O
        self.n = len(O) - 1 # check that 
        self.Theta = Theta
        self.p = p
        self.o_XY = o_XY
        self.v_XY = v_XY
        self.g = g
        self.u = u
        self.L = L
        self.CB = CB

    def search1(self, X, Y, v, d):
        """
        Perform the SEARCH1 algorithm.
        """
        if d == self.n + 1: # end of the tree
            self.CB = X # update the current best X solution (here there is not the check that the best solution is better than the current one. No problem, it there is one before calling the function search)
            self.L = v # update the current best value of X
        else:            
            if self.v_XY(union_listOflists_list(X,self.O[d]), Y, self.O, self.Theta, self.p, self.g, self.u) > self.L:
                self.search1(union_listOflists_list(X,self.O[d]), Y, self.v_XY(union_listOflists_list(X,self.O[d]), Y, self.O, self.Theta, self.p, self.g, self.u), d + 1)

            if self.v_XY(X, union_listOflists_list(Y,self.O[d]), self.O, self.Theta, self.p, self.g, self.u) > self.L:
                self.search1(X,union_listOflists_list(Y,self.O[d]), self.v_XY(X, union_listOflists_list(Y,self.O[d]), self.O, self.Theta, self.p, self.g, self.u), d + 1)          


In [78]:
def branch_and_bound_ds(O, Theta, p, o_XY, v_XY, g, u):
    start_time = time.time()  # Record the start time

    CB = None # initilize the current best solution
    L = float('-inf')
    
    SearchAlgorithm_instance = SearchAlgorithm(O, Theta, p, o_XY, v_XY, g, u, L, CB)
    SearchAlgorithm_instance.search1([], [], 0, 1)
    CB = SearchAlgorithm_instance.CB

    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate the elapsed time

    return CB, round(elapsed_time,2)

# Application 

## 1: No IR - exp 1 and exp2

Classes

In [79]:
class g_class:
    def __init__(self, Theta, O, minimum_value = 0, maximum_value = 100):
        self.Theta = Theta  # A list of elements in Theta
        self.O = O          # A list of lists
        self.values = {}
        for theta in Theta:
            for o in O:
                o_tuple = tuple(o)  # Convert the list o to a tuple for dictionary key
                self.values[(theta, o_tuple)] = random.randint(minimum_value, maximum_value)

    def g(self, theta, o):
        o_tuple = tuple(o)  # Convert the list o to a tuple for lookup
        if (theta, o_tuple) in self.values:
            return self.values[(theta, o_tuple)]
        else:
            raise ValueError("The given theta and o pair is not in the initialized set.")


In [80]:
class u_class:
    def __init__(self, Theta, O, minimum_value = 0, maximum_value = 100):
        self.Theta = Theta  # A list of elements in Theta
        self.O = O          # A list of lists
        self.values = {}
        for theta in Theta:
            for o in O:
                o_tuple = tuple(o)  # Convert the list o to a tuple for dictionary key
                self.values[(theta, o_tuple)] = random.randint(minimum_value, maximum_value)

    def u(self, theta, o):
        o_tuple = tuple(o)  # Convert the list o to a tuple for lookup
        if (theta, o_tuple) in self.values:
            return self.values[(theta, o_tuple)]
        else:
            raise ValueError("The given theta and o pair is not in the initialized set.")

Check IR constraint

In [81]:
# It checks if, for all theta in Theta, there is at least one o in O s.t. u(theta, o) >= 0
# If the above condition is true, it returns True, otherwise False
def check_IR(Theta, X, u):
    check_list = [False for _ in range(len(X))]
    for theta in Theta:
        i = 0
        for o in X:
            if u(theta, o) >= 0:
                check_list[i] = True
            i += 1
    return all(check_list)


# An example 
O = [[i] for i in range(10)]
X_bar = random.sample(O, 5)
Theta = [i for i in range(20)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = g_class(Theta, O, minimum_value=-50, maximum_value=5)
u_instance = u_class(Theta, O, minimum_value=-50, maximum_value=5) 
print(
    check_IR(Theta=Theta, X=X_bar, u=u_instance.u)
    )

True


An instance

In [82]:
# An instance
O = [[i] for i in range(20)]
Theta = [i for i in range(10)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = g_class(Theta, O, minimum_value=0, maximum_value=100)
u_instance = u_class(Theta, O, minimum_value=0, maximum_value=100) 


Find X

In [83]:
def find_X(O, Theta, p, o_XY, v_XY, g_instance, u_instance):
    CB, elapsed_time = branch_and_bound_ds(O, Theta, p, o_XY, v_XY, g_instance.g, u_instance.u)
    model_vX_value = v_X(CB, Theta, p, g_instance.g, u_instance.u)

    semi_random_CB = random.sample(O, len(CB))
    semi_random_vX_value = v_X(semi_random_CB, Theta, p, g_instance.g, u_instance.u)
    if model_vX_value != 0:
        model_vs_rand = round((model_vX_value-semi_random_vX_value)/model_vX_value * 100,2)
    else:
        model_vs_rand = None

    return CB, round(model_vX_value,2), elapsed_time, semi_random_CB, round(semi_random_vX_value,2), model_vs_rand

CB, model_vX_value, elapsed_time, semi_random_CB, semi_random_vX_value, model_vs_rand = find_X(O, Theta, p, o_XY, v_XY, g_instance, u_instance)

print("Check IR for the model", check_IR(Theta=Theta, X=CB, u=u_instance.u))
print("Check IR for the semi-rnd", check_IR(Theta=Theta, X=semi_random_CB, u=u_instance.u))
      
print("\nElapsed time = ", elapsed_time)
print("Model X =", CB)
print("Semi random model X = ",semi_random_CB)
print("\nModel vX value ", model_vX_value)
print("Semi random model vX value ", semi_random_vX_value)
print("Relative improvement of the model ", model_vs_rand," %")

Check IR for the model True
Check IR for the semi-rnd True

Elapsed time =  1.79
Model X = [[12], [18], [2], [15], [11], [8], [14], [4], [17], [19], [9]]
Semi random model X =  [[12], [14], [5], [1], [3], [0], [10], [2], [11], [13], [16]]

Model vX value  82.9
Semi random model vX value  54.4
Relative improvement of the model  34.38  %


Check g and u of the mean of all the types 

In [84]:
# Check the model on a single type

def check_model_results(CB, g_instance, u_instance, Theta, semi_random_CB):
    outcomeForAllThetas_model = [o_X(theta = theta, X = CB, g = g_instance.g, u = u_instance.u) for theta in Theta]
    gForAllOutcomes_model = [g_instance.g(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]
    uForAllOutcomes_model = [u_instance.u(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]

    outcomeForAllThetas_semiRandom = [o_X(theta = theta, X = semi_random_CB, g = g_instance.g, u = u_instance.u) for theta in Theta]
    gForAllOutcomes_semiRandom = [g_instance.g(theta=theta, o=o_semiRandom) for theta, o_semiRandom in zip(Theta, outcomeForAllThetas_semiRandom)]
    uForAllOutcomes_semiRandom = [u_instance.u(theta=theta, o=o_semiRandom) for theta, o_semiRandom in zip(Theta, outcomeForAllThetas_semiRandom)]

    return gForAllOutcomes_model, uForAllOutcomes_model, gForAllOutcomes_semiRandom, uForAllOutcomes_semiRandom



gForAllOutcomes_model, uForAllOutcomes_model, gForAllOutcomes_semiRandom, uForAllOutcomes_semiRandom = check_model_results(CB, g_instance, u_instance, Theta, semi_random_CB)

print("        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(gForAllOutcomes_semiRandom),2), round(np.mean(uForAllOutcomes_semiRandom),2))

         g   u
model    82.9 88.6
semi rnd 54.4 92.0


## 2 - IR: exp 1 and exp 2

An instance

In [85]:
# An instance
O = [[i] for i in range(10)]
Theta = [i for i in range(20)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = g_class(Theta, O, minimum_value=-50, maximum_value=50)
u_instance = u_class(Theta, O, minimum_value=-50, maximum_value=50) 


Find X

In [86]:
CB, model_vX_value, elapsed_time, semi_random_CB, semi_random_vX_value, model_vs_rand  = find_X(O, Theta, p, o_XY, v_XY, g_instance, u_instance)

print("Check IR for the model", check_IR(Theta=Theta, X=CB, u=u_instance.u))
print("Check IR for the semi-rnd", check_IR(Theta=Theta, X=semi_random_CB, u=u_instance.u))
print("\nElapsed time = ", elapsed_time)
print("Model X =", CB)
print("Semi random model X = ",semi_random_CB)
print("\nModel vX value ", model_vX_value)
print("Semi random model vX value ", semi_random_vX_value)
print("Relative improvement of the model ", model_vs_rand," %")

Check IR for the model True
Check IR for the semi-rnd True

Elapsed time =  0.31
Model X = [[2], [3]]
Semi random model X =  [[8], [5]]

Model vX value  12.2
Semi random model vX value  -9.95
Relative improvement of the model  181.56  %


Check g and u of the mean of all the types 

In [87]:
# Check g and u
gForAllOutcomes_model, uForAllOutcomes_model, gForAllOutcomes_semiRandom, uForAllOutcomes_semiRandom = check_model_results(CB, g_instance, u_instance, Theta, semi_random_CB)

print("        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(gForAllOutcomes_semiRandom),2), round(np.mean(uForAllOutcomes_semiRandom),2))

         g   u
model    12.2 16.45
semi rnd -9.95 17.6


## 3 - Bartering

In [88]:
class g_bartering_class:
    def __init__(self, Theta, O, minimum_value = 0, maximum_value = 10):
        self.Theta = Theta  # A list of elements in Theta
        self.O = O          # A list of lists
        self.values = {}
        self.outcome_size = len(O[0])
        for i in range(self.outcome_size):
            self.values[(i)] = random.randint(minimum_value, maximum_value)

    def g(self,theta, o): # in this specific case, g is indepenedent from theta
        return sum(self.values[i]*(1-o[i]) for i in range(self.outcome_size)) # o[i] = 0 if designer took the item i, 1 otherwise


In [89]:
class u_bartering_class:
    def __init__(self, Theta, O, minimum_value = 0, maximum_value = 10):
        self.Theta = Theta  # A list of elements in Theta
        self.O = O          # A list of lists
        self.values = {}
        self.outcome_size = len(O[0])
        for theta in Theta:
            for i in range(self.outcome_size):
                self.values[(theta, i)] = random.randint(minimum_value, maximum_value)

    def u(self, theta, o):
        return sum(self.values[(theta,i)]*(o[i]) for i in range(self.outcome_size)) # o[i] = 1 if agent took the item i, 0 otherwise


An instance 

In [90]:
# An instance
number_of_items = 10
number_of_outcomes = 20
number_of_types = 5
minimum_value = 0 # try -5
maximum_value = 10 # try 5


all_possible_outcomes = list(itertools.product([0, 1], repeat=number_of_items))
O = random.sample(all_possible_outcomes, number_of_outcomes)
Theta = [i for i in range(number_of_types)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = g_bartering_class(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value)
u_instance = u_bartering_class(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value) 


Find X

In [91]:
CB, model_vX_value, elapsed_time, semi_random_CB, semi_random_vX_value, model_vs_rand = find_X(O, Theta, p, o_XY, v_XY, g_instance, u_instance)

print("Check IR for the model", check_IR(Theta=Theta, X=CB, u=u_instance.u))
print("Check IR for the semi-rnd", check_IR(Theta=Theta, X=semi_random_CB, u=u_instance.u))
print("\nElapsed time = ", elapsed_time)
print("Model X =", CB)
print("Semi random model X = ",semi_random_CB)
print("\nModel vX value ", model_vX_value)
print("Semi random model vX value ", semi_random_vX_value)
print("Relative improvement of the model ", model_vs_rand," %")

Check IR for the model True
Check IR for the semi-rnd True

Elapsed time =  3.2
Model X = [[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]]
Semi random model X =  [(1, 0, 1, 1, 0, 0, 1, 1, 0, 1)]

Model vX value  36.0
Semi random model vX value  17.0
Relative improvement of the model  52.78  %


Check g and u on the mean if all possible theta

In [92]:
# Check g and u
gForAllOutcomes_model, uForAllOutcomes_model, gForAllOutcomes_semiRandom, uForAllOutcomes_semiRandom = check_model_results(CB, g_instance, u_instance, Theta, semi_random_CB)

print("        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(gForAllOutcomes_semiRandom),2), round(np.mean(uForAllOutcomes_semiRandom),2))

         g   u
model    36.0 10.0
semi rnd 17.0 33.2


# Extra things

## Another utility tipe for the bartering problem 

In [93]:
class g_bartering_class_difference:
    def __init__(self, Theta, O, minimum_value = 0, maximum_value = 10, initial_goods = None):
        self.Theta = Theta  # A list of elements in Theta
        self.O = O          # A list of lists
        self.values = {}
        self.outcome_size = len(O[0])
        self.initial_goods = initial_goods
        for i in range(self.outcome_size):
            self.values[(i)] = random.randint(minimum_value, maximum_value)
        self.initial_values = sum(self.values[i]*(1-self.initial_goods[i]) for i in range(self.outcome_size))
        
    def g(self,theta, o): # in this specific case, g is indepenedent from theta
        current_value = sum(self.values[i]*(1-o[i]) for i in range(self.outcome_size)) 
        return current_value - self.initial_values 


In [94]:
class u_bartering_class_difference:
    def __init__(self, Theta, O, minimum_value = 0, maximum_value = 10, initial_goods = None):
        self.Theta = Theta  # A list of elements in Theta
        self.O = O          # A list of lists
        self.values = {}
        self.initial_values = {}
        self.outcome_size = len(O[0])
        self.initial_goods = initial_goods  
        for theta in Theta:
            for i in range(self.outcome_size):
                self.values[(theta, i)] = random.randint(minimum_value, maximum_value)
        for theta in Theta:
            self.initial_values[theta] = sum(self.values[(theta,i)]*(self.initial_goods[i]) for i in range(self.outcome_size))

    def u(self, theta, o):
        current_value = sum(self.values[(theta,i)]*(o[i]) for i in range(self.outcome_size)) # o[i] = 1 if agent took the item i, 0 otherwise
        return current_value - self.initial_values[theta]


In [95]:
# An instance
number_of_items = 8
number_of_outcomes = 30
number_of_types = 5
minimum_value = 0 # with [0,10] the IR constraint is mainly = False 
maximum_value = 10

all_possible_outcomes = list(itertools.product([0, 1], repeat=number_of_items))
O = random.sample(all_possible_outcomes, number_of_outcomes)
initial_goods = random.choice(O)
Theta = [i for i in range(number_of_types)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = g_bartering_class_difference(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value, initial_goods=initial_goods)
u_instance = u_bartering_class_difference(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value, initial_goods=initial_goods) 



In [96]:
CB, model_vX_value, elapsed_time, semi_random_CB, semi_random_vX_value, model_vs_rand = find_X(O, Theta, p, o_XY, v_XY, g_instance, u_instance)

print("Check IR for the model", check_IR(Theta=Theta, X=CB, u=u_instance.u))
print("Check IR for the semi-rnd", check_IR(Theta=Theta, X=semi_random_CB, u=u_instance.u))
print("\nElapsed time = ", elapsed_time)
print("Model X =", CB)
print("Semi random model X = ",semi_random_CB)
print("\nModel vX value ", model_vX_value)
print("Semi random model vX value ", semi_random_vX_value)
print("Relative improvement of the model ", model_vs_rand," %")

Check IR for the model False
Check IR for the semi-rnd False

Elapsed time =  15.83
Model X = [[1, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 1], [1, 0, 0, 0, 1, 1, 0, 0]]
Semi random model X =  [(1, 1, 0, 0, 1, 0, 0, 0), (0, 1, 0, 0, 1, 1, 1, 1), (1, 0, 1, 0, 1, 1, 1, 0)]

Model vX value  44.0
Semi random model vX value  21.4
Relative improvement of the model  51.36  %


In [97]:
# Check g and u
gForAllOutcomes_model, uForAllOutcomes_model, gForAllOutcomes_semiRandom, uForAllOutcomes_semiRandom = check_model_results(CB, g_instance, u_instance, Theta, semi_random_CB)

print("        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(gForAllOutcomes_semiRandom),2), round(np.mean(uForAllOutcomes_semiRandom),2))


         g   u
model    44.0 -26.4
semi rnd 21.4 -10.8


### The IR constraint 
The experiment with the bartering problem where the utility and the objective are the difference from the initial values of the goods and the final one could be harder, because we are sure there exist cases with a negative utility/objective. 
While, this could not happen in the case where the utiliy is the sum of the values of the goods and the values are all non negative.

I did some experiments with a configuration I typically used in the previous cases. The brench and bound model with had a v_x really better then the random model. 
However, I got a bad result. 
I saw that, altough the objective was very high, the mean of the utilities was negative. 
In that case, the model makes no sense, beacuse anyone would choose to play against a designer who make you a negative utility at the end of the game (it means that the agent values the goods that it has at the end less then the goods that it had at th beginning). 

So that, I checked the IR constraint and I was that it was not respected. 
It makes sense, beacuse there is a Lemma that asserts that $o_X$ is IR if and only for all types $theta$ exists an outcome $o \in O$ s.t. the utility is non-negative.

Then, I changed instance of the problem in terms of parameter. 
Expecially, I increase as more as my PC allowed me (in terms of computation time) the size of the outcome, in order to increase the probbility that this constraint holds. 
Finally, I get an instance where the IR constraint holds and that was the result: a positive utility!
So, the model could be used. 
Moreover, it improve the objective very well.


With the following configuration
```python
number_of_items = 10
number_of_outcomes = 30
number_of_types = 1
minimum_value = 0 
maximum_value = 10
```
the IR constraint holds indeed we get 
```
Check IR for the model True
Check IR for the semi-rnd True

Elapsed time =  0.58
Model X = [[0, 1, 1, 1, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 1, 1, 1, 1, 0], [0, 1, 1, 0, 1, 0, 0, 0, 1, 0], [0, 1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 1, 0, 1, 1, 0, 0, 0, 1], [0, 1, 0, 0, 1, 1, 1, 0, 1, 0], [0, 0, 0, 0, 1, 1, 0, 0, 1, 1], [0, 1, 0, 0, 0, 0, 1, 0, 1, 1]]
Semi random model X =  [(1, 1, 0, 0, 0, 1, 1, 0, 1, 0), (0, 1, 1, 0, 0, 0, 1, 1, 1, 0), (0, 1, 0, 0, 1, 1, 1, 0, 1, 0), (0, 0, 1, 0, 1, 1, 0, 0, 0, 1), (0, 1, 0, 0, 1, 0, 0, 1, 1, 1), (0, 0, 0, 1, 1, 0, 0, 1, 1, 1), (1, 0, 1, 1, 1, 1, 0, 1, 1, 0), (1, 1, 0, 1, 1, 0, 0, 0, 0, 0)]

Model vX value  16.0
Semi random model vX value  -9.0
Relative improvement of the model  156.25  %
```
and values 
```
         g   u
model    16.0 4.0
semi rnd -9.0 23.0
```

